In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv')

In [ ]:
df.head()

In [ ]:
list(df)

In [ ]:
test = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')

In [ ]:
test.head()

In [ ]:
shops = pd.read_csv('../input/competitive-data-science-predict-future-sales/shops.csv')

In [ ]:
shops.head()

In [ ]:
items = pd.read_csv('../input/competitive-data-science-predict-future-sales/items.csv')

In [ ]:
items.head()

In [ ]:
item_cats = pd.read_csv('../input/competitive-data-science-predict-future-sales/item_categories.csv')
item_cats.head()

In [ ]:
test.set_index('ID',inplace=True)

test = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')
shops = pd.read_csv('../input/competitive-data-science-predict-future-sales/shops.csv')
items = pd.read_csv('../input/competitive-data-science-predict-future-sales/items.csv')
item_cats = pd.read_csv('../input/competitive-data-science-predict-future-sales/item_categories.csv')

## test.set_index('ID',inplace=True)

In [ ]:
test

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
shops.shape

In [ ]:
shops.columns

In [ ]:
shops.set_index('shop_id',inplace=True)

In [ ]:
shops.columns

In [ ]:
df['shop_id'].nunique()

In [ ]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month

In [ ]:
df.columns

In [ ]:
item_cats.set_index('item_category_id',inplace=True)

In [ ]:
item_cats

In [ ]:
items.set_index('item_id',inplace=True)

In [ ]:
items

In [ ]:
items['item_name'].nunique()

In [ ]:
no_item_id = [i for i in range(22170)]

In [ ]:
for i in no_item_id:
    if len(df[df['item_id']==i]['item_price'])==0:
        print(i,'-->',0)
    else:
        print(i,'-->',df[df['item_id']==i]['item_price'].iat[0])
# df[df['item_id']==no_item_id[0]]['item_price'].iat[0]

In [ ]:
item_id_and_price = {}
for i in no_item_id:
    if len(df[df['item_id']==i]['item_price'])==0:
        item_id_and_price[i] = 0
    else:
        item_id_and_price[i] = df[df['item_id']==i]['item_price'].iat[0]

In [ ]:
test

In [ ]:
item_id_and_price = pd.Series(item_id_and_price)

In [ ]:
test.info()

In [ ]:
def get_price(x):
    return item_id_and_price.get(x,default='data not available')
test['item_price'] = test['item_id'].apply(get_price)

In [ ]:
test

In [ ]:
df

In [ ]:
df.groupby(['year','month']).mean().loc[:,'item_price':'item_cnt_day']

From the groupby call we can clearly see that 'item_cnt_day' for a month of a year is higher than that of the previous year

In [ ]:
plt.figure(figsize=(12,8),dpi=150)
df['date_block_num'].value_counts().plot(kind='bar');

In [ ]:
plt.figure(figsize=(12,8),dpi=150)
df['month'].value_counts().plot(kind='bar');

In [ ]:
plt.figure(figsize=(12,8),dpi=150)
sns.countplot(x='month',data=df,hue='year')
plt.legend(loc='best');

We can see that sales has been pretty low for the 2015 and sales has been reducing in a yearly basis.

Russian seasons<br>
Spring - March, April, May<br>
Summer - June to August<br>
Autumn - Sept to Nov<br>
Winter - Dec to Feb

In [ ]:
def seasons(x):
    if x in [3,4,5]:
        return 'spring'
    elif x in [6,7,8]:
        return 'summer'
    elif x in [9,10,11]:
        return 'autumn'
    else:
        return 'winter'
df['season'] = df['month'].apply(seasons)

In [ ]:
df

In [ ]:
plt.figure(figsize=(12,4),dpi=150)
sns.countplot(x='season',data=df,hue='year')
plt.legend(loc='best');

In [ ]:
plt.figure(figsize=(12,8),dpi=150)
sns.boxplot(y='item_cnt_day',x='season',hue='year',data=df)

Let's remove the outliers

In [ ]:
df.drop(df[df['season']=='autumn']['item_cnt_day'].idxmax(),inplace=True)
df.drop(df[df['season']=='winter']['item_cnt_day'].idxmax(),inplace=True)

After removing the outliers

In [ ]:
plt.figure(figsize=(12,8),dpi=150)
sns.boxplot(y='item_cnt_day',x='season',hue='year',data=df);

In [ ]:
df['item_cnt_day'].describe()

In [ ]:
test

In [ ]:
df['shop_id'].nunique()

In [ ]:
test['shop_id'].nunique()

In [ ]:
df.groupby('shop_id').mean().loc[:,['item_cnt_day','date_block_num','item_price']]

In [ ]:
shop_mean = df.groupby('shop_id').mean().loc[:,['item_cnt_day','date_block_num','item_price']]

In [ ]:
shop_mean

In [ ]:
test

In [ ]:
def mean_price(x):
    return shop_mean.iat[x,2]
test['mean_price_for_shop'] = test['shop_id'].apply(mean_price)

In [ ]:
def block_num(x):
    return shop_mean.iat[x,1]
test['mean_date_block_num'] = test['shop_id'].apply(block_num)

In [ ]:
test

Encoding the 'mean_price_for_shop' column.

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
test['mean_price_for_shop'] = le.fit_transform(test['mean_price_for_shop'])

In [ ]:
le = LabelEncoder()
test['mean_date_block_num'] = le.fit_transform(test['mean_date_block_num'])

In [ ]:
test

In [ ]:
df.drop('date',axis=1,inplace=True)

In [ ]:
df

In [ ]:
plt.figure(figsize=(12,8),dpi=150)
sns.boxplot(x='season',y='item_price',data=df,hue='year'); 

Another outlier

In [ ]:
df[df['season']=='winter']['item_price'].idxmax()

In [ ]:
df.drop(df[df['season']=='winter']['item_price'].idxmax(),inplace=True)

In [ ]:
plt.figure(figsize=(12,8),dpi=150)
sns.boxplot(x='season',y='item_price',data=df,hue='year'); 

In [ ]:
df['item_price'].min(),df['item_price'].max()

In [ ]:
df[df['item_price']<0]

In [ ]:
df.drop(df[df['item_price']<0].index,inplace=True)

In [ ]:
df['item_price'].min(),df['item_price'].max()

In [ ]:
df['item_price'].describe().iloc[1:]

In [ ]:
def item_price_range(x):
    if 0<=x<249:
        return 'First'
    elif 249<=x<=399:
        return 'Second'
    elif 399<x<=999:
        return 'Third'
    else:
        return 'Fourth'
df['price_range_quartile'] = df['item_price'].apply(item_price_range)
test['price_range_quartile'] = test['item_price'].apply(item_price_range)

In [ ]:
df

In [ ]:
sns.countplot(x='price_range_quartile',data=df,hue='year')

In [ ]:
test

In [ ]:
df

In [ ]:
df.corr()

In [ ]:
df.groupby('year').mean()

In [ ]:
test

In [ ]:
def artificial_year(x):
    if x<5.52:
        return 2013
    elif x<17.60:
        return 2014
    else:
        return 2015
test['year'] = test['mean_date_block_num'].apply(artificial_year)

In [ ]:
test

In [ ]:
plt.figure(figsize=(12,8))
sns.countplot(x='price_range_quartile',data=df,hue='year')

In [ ]:
df.head(2)

In [ ]:
test.columns

In [ ]:
test1 = test[['mean_date_block_num','shop_id','item_id','item_price','year','price_range_quartile','mean_price_for_shop']]

In [ ]:
test1.head(2)

In [ ]:
df.head(2)

In [ ]:
def mean_price(x):
    return shop_mean.iat[x,2]
df['mean_price_for_shop'] = df['shop_id'].apply(mean_price)

In [ ]:
le = LabelEncoder()
df['mean_price_for_shop'] = le.fit_transform(df['mean_price_for_shop'])

In [ ]:
df.head(2)

In [ ]:
test1.head(2)

In [ ]:
sns.countplot(y='season',data=df)

In [ ]:
df.drop(['season','month'],axis=1,inplace=True)

In [ ]:
df

In [ ]:
test1

In [ ]:
df['year'] = df['year'].astype(str)
test1['year'] = test1['year'].astype(str)

In [ ]:
df_dummy = pd.get_dummies(df[['year','price_range_quartile']],drop_first=True)
test1_dummy = pd.get_dummies(test1[['year','price_range_quartile']],drop_first=True)

In [ ]:
df = pd.concat([df,df_dummy],axis=1)
test1 = pd.concat([test1,test1_dummy],axis=1)

In [ ]:
df.drop('year',axis=1,inplace=True)
test1.drop('year',axis=1,inplace=True)

In [ ]:
df.drop('price_range_quartile',axis=1,inplace=True)
test1.drop('price_range_quartile',axis=1,inplace=True)

In [ ]:
df.head(2)

In [ ]:
# df.loc[:,'date_block_num':'year_2015']
df = df.iloc[:,:11]

In [ ]:
X = df.drop('item_cnt_day',axis=1)
y = df['item_cnt_day']

In [ ]:
X.head(2)

In [ ]:
test1.head(2)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor

In [ ]:
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=101)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Base model with no hyperparameter tuning

In [ ]:
models = [LinearRegression(),Lasso(),Ridge(),KNeighborsRegressor(),RandomForestRegressor()]
data = {}
for m in models:
    m.fit(X_train,y_train)
    pred = m.predict(X_test)
    data[str(m)] = [mean_absolute_error(y_test,pred),np.sqrt(mean_squared_error(y_test,pred))]
model_df = pd.DataFrame(data)

In [ ]:
model_df

In [ ]:
final_model = make_pipeline(StandardScaler(),RandomForestRegressor())

In [ ]:
final_model.fit(X,y)

In [ ]:
item_cnt_pred = final_model.predict(test1)

In [ ]:
item_cnt_pred

In [ ]:
test1.drop(test1.columns,axis=1,inplace=True)

In [ ]:
test1['item_cnt_month'] = item_cnt_pred

In [ ]:
test1

In [ ]:
test1.to_csv('submissions.csv')